In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import math

# **Pre-Processing**


These cells clean text files and generate csv's for Test and Train data.Don't run these if you already have csv's

In [74]:
#Preprocessing
!pip install Urduhack[tf] 
import urduhack 
urduhack.download()
import re
from urduhack.preprocessing import normalize_whitespace
from urduhack.preprocessing import remove_punctuation

In [75]:
def clean(data):
  text = remove_punctuation(data)
  text=re.sub(r"[A-Za-z0-9@;:]", "", text, flags=re.UNICODE)
  text=text.replace("\n ", ".")
  text=text.replace("\n  ", ".")
  text=text.replace("\n", ".")
  text = re.sub(r'\.+', "\n", text)
  text = normalize_whitespace(text)
  text = remove_punctuation(text)
  text=text.replace("\n", " ")
  text = normalize_whitespace(text)
  return text

Test_Fake=[]
Test_Real=[]
Train_Fake=[]
Train_Real=[]
for i in range (1,351):
  with open('/content/drive/MyDrive/NLP5_Data/Train/Real/R ('+str(i)+').txt','r') as f:
    text = f.read().replace('\n', '')
    Train_Real.append(clean(text)) 
for i in range (1,289):
  with open('/content/drive/MyDrive/NLP5_Data/Train/Fake/F ('+str(i)+').txt','r') as f:
    text = f.read().replace('\n', '')
    Train_Fake.append(clean(text))
Train = Train_Fake+Train_Real
L1 = [0] * len(Train_Fake)
L2 = [1] * len(Train_Real)
Class=L1+L2
di = {'Text':Train,'Class':Class}
df = pd.DataFrame(di)
df.to_csv("/content/drive/MyDrive/NLP5_Data/Train.csv",index=False)

for i in range (1,151):
  with open('/content/drive/MyDrive/NLP5_Data/Test/Real/R ('+str(i)+').txt','r') as f:
    text = f.read().replace('\n', '')
    Test_Real.append(clean(text))
for i in range (1,113):
  with open('/content/drive/MyDrive/NLP5_Data/Test/Fake/F ('+str(i)+').txt','r') as f:
    text = f.read().replace('\n', '')
    Test_Fake.append(clean(text))
Test = Test_Fake + Test_Real
L1 = [0] * len(Test_Fake)
L2 = [1] * len(Test_Real)
Class=L1+L2
di = {'Text':Test,'Class':Class}
df = pd.DataFrame(di)
df.to_csv("/content/drive/MyDrive/NLP5_Data/Test.csv",index=False)

# **Loading CSV's**


Loading csv's

In [79]:
df_Train=pd.read_csv('/content/drive/MyDrive/NLP5_Data/Train.csv')
df_Test=pd.read_csv('/content/drive/MyDrive/NLP5_Data/Test.csv')

In [80]:
with open('/content/drive/MyDrive/NLP5_Data/stopwords-ur.txt','r') as f:
  stopwords = f.read().splitlines()

In [81]:
Train = df_Train["Text"].to_numpy()
Class_Train = df_Train["Class"].to_numpy()
Test = df_Test["Text"].to_numpy()
Class_Test= df_Test["Class"].to_numpy()

# **Training**


In [90]:
def RemoveStopwords(Text):
  StopwordText=[]
  for sentence in Text:
    StopwordText.append( " ".join( [wrd for wrd in sentence.split() if wrd not in stopwords]) )
  return np.array(StopwordText)

def TrainMultinomialNB(Text,Class):
  V=Counter(( " ".join(Text) ).split())
  N=len(Text)
  C=np.unique(Class)
  prior={}
  condprob={}
  for c in C: 
    mask = Class==c
    doc = Text[mask]  #Takes all texts of class c 
    Nc=len(doc)
    prior[c]=Nc/N   #  priors{c1:P(c1), c2:P(c2) }  stores classes and their priors in dictionary
    counts=Counter(wrd for txt in doc for wrd in txt.split()) #calculates counts of each token in doc
    Nw = sum(counts.values())#number of texts in Class c 
    for w in V:
      Ni=counts[w]  # count of word w in documents 
      p=(Ni+1)/(Nw + len(V))
      condprob.setdefault(c,{})[w] = p  # condprob{ c1:{w1:P(w1|c1)}, c1:{w2:P(w2|c1)}...  }  stores class, word and conditional prob in dictionary
    

  return V, prior, condprob

def TrainMultinomialNB_Boolean(Text,Class):
  V=Counter(( " ".join(Text) ).split())
  N=len(Text)
  C=np.unique(Class)
  prior={}
  condprob={}
  for c in C:
    mask = Class==c
    TXTc = Text[mask]  #Takes all texts of class c 
    Nc=len(TXTc)
    prior[c]=Nc/N   #  priors{c1:P(c1), c2:P(c2) }  stores classes and their priors in dictionary
    doc=[  " ".join(Counter(txt.split()).keys())  for txt in TXTc  ]  # Remove duplicates  TXTc 
    counts=Counter(wrd for txt in doc for wrd in txt.split()) #calculates counts of each token in doc
    Nw = sum(counts.values())#number of texts in Class c
    for w in V:
      Ni=counts[w]  # count of word w in documents
      p=(Ni+1)/(Nw + len(V))
      condprob.setdefault(c, {})[w] = p  # condprob{ c1:{w1:P(w1|c1)}, c1:{w2:P(w2|c1)}...  }  stores class, word and conditional prob in dictionary
  return V, prior, condprob
  

def ApplyMultinomialNB(C, V, prior, condprob, t,isBool):
  if isBool:
    t=[  " ".join(Counter(txt.split()).keys())  for txt in [t]  ]  # Remove duplicates from test set
  tokens=( " ".join(t) ).split() #covert list to  string, split on space
  W= [w for w in tokens if w in V ] 
  score={}
  for  c in C:
    # print('hereC',c ,end=' , ' )
    score[c] = math.log(prior[c])
    for  w in W:
      score[c]+= math.log(condprob[c][w])  
  
  return max(score,key=score.get)  

# **Testing**


In [61]:
# Train= np.array(['chineese beijin chineese','chineese shanghai chineese','chineese food','tokyo japan'] )
# Class_Train= np.array([0,0,0,1])
# Test= np.array(['chineese chineese chineese tokyo','chineese shanghai chineese'] )
# Class_Test= np.array([0,1])

In [91]:
C=[0,1]  #unique set of all classes
Test_S = RemoveStopwords(Test)
Train_S = RemoveStopwords(Train)

#- Naive Bayes without stopwords
V , prior, condprob = TrainMultinomialNB(Train,Class_Train)
NB=[ ApplyMultinomialNB(C, V, prior, condprob, txt,False) for txt in Test]
print("NB ",NB)

#- Naive Bayes with stopwords
V , prior, condprob = TrainMultinomialNB(Train_S ,Class_Train)
NB_S=[ ApplyMultinomialNB(C, V, prior, condprob, txt,False) for txt in Test_S]
print("NB_S ",NB_S)

#- Boolean Naive Bayes without stopwords
V, prior, condprob = TrainMultinomialNB_Boolean(Train ,Class_Train)
NBB=[ ApplyMultinomialNB(C, V, prior, condprob, txt,True) for txt in Test]
print("NBB ",NBB)

#--Boolean Naive Bayes with stopwords
V , prior, condprob = TrainMultinomialNB_Boolean(Train_S ,Class_Train)
NBB_S=[ ApplyMultinomialNB(C, V, prior, condprob, txt,True) for txt in Test_S]
print("NBB_S ",NBB_S)

NB  [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]
NB_S  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

# **..................................................... REPORT ......................................................**

****

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [92]:
print('Accuracy score\n   NB without stopwords :', accuracy_score(Class_Test,NB),'\n   NB with stopwords :', accuracy_score(Class_Test,NB_S))
print('   NB Boolean without stopwords :', accuracy_score(Class_Test,NBB),'\n   NB Boolean with stopwords :', accuracy_score(Class_Test,NBB_S))

print('Precision score\n   NB without stopwords :', precision_score(Class_Test,   NB),'\n   NB with stopwords :', precision_score(Class_Test,NB_S))
print('    NB Boolean without stopwords :', precision_score(Class_Test,NBB),'\n   NB Boolean with stopwords :', precision_score(Class_Test,NBB_S))

print('Recall score\n   NB without stopwords :', recall_score(Class_Test,   NB),'\n   NB with stopwords :', recall_score(Class_Test,NB_S))
print('   NB Boolean without stopwords :', recall_score(Class_Test,NBB),'\n   NB Boolean with stopwords :', recall_score(Class_Test,NBB_S))

print('F1 score\n   NB without stopwords :', f1_score(Class_Test,NB),'\n   NB with stopwords :', f1_score(Class_Test,   NB_S))
print('   NB Boolean without stopwords :', f1_score(Class_Test,NBB),'\n   NB Boolean with stopwords :', f1_score(Class_Test,NBB_S))

Accuracy score
   NB without stopwords : 0.5763358778625954 
   NB with stopwords : 0.5725190839694656
   NB Boolean without stopwords : 0.7633587786259542 
   NB Boolean with stopwords : 0.7633587786259542
Precision score
   NB without stopwords : 0.5859030837004405 
   NB with stopwords : 0.5725190839694656
    NB Boolean without stopwords : 0.7619047619047619 
   NB Boolean with stopwords : 0.7650602409638554
Recall score
   NB without stopwords : 0.8866666666666667 
   NB with stopwords : 1.0
   NB Boolean without stopwords : 0.8533333333333334 
   NB Boolean with stopwords : 0.8466666666666667
F1 score
   NB without stopwords : 0.7055702917771883 
   NB with stopwords : 0.7281553398058253
   NB Boolean without stopwords : 0.8050314465408804 
   NB Boolean with stopwords : 0.8037974683544303


It is evident from the accuarcies above that removal of stopwords brings a veriy significant improvement in the results.And removing duplicates i.e Boolean NB gives best results of the four methods